## Regression with Scikit-Learn

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

In [2]:
from sklearn.datasets import fetch_openml
boston = fetch_openml(data_id=43465)
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df[boston.target_names[0]] = boston.target

In [3]:
#Check dataframe
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [4]:
#Normalize the z-score for all columns because some columns show signs of having different units and are not on the same scale.
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df

NameError: name 'StandardScaler' is not defined

## Task 1: Data Splitting Strategies

### Task 1.a.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Plotting the relationship between features using a Pairplot to initially detect correlations
sns.pairplot(df)
plt.show()

In [ ]:
#Plotting a heatmap with correlation coefficients to check the degree of correlation between features.
corr = df.corr()
sns.heatmap(corr, annot=True, fmt=" .1f")
plt.show()

**Given the dataset size (506 rows) and number of features, K-Fold Cross Validation is chosen for training and validation to ensure accuracy and stability. The model will then be evaluated using performance metrics and visualized to show the error between the test and predicted sets.**

In [ ]:
# Initializing KFold with 5 splits and shuffling the data before splitting
from sklearn.model_selection import KFold
kfold = KFold(n_splits = 5, shuffle=True)

In [ ]:
# Splitting the dataset into training and testing sets, with 30% for testing
from sklearn.model_selection import train_test_split
x = df.drop(columns=['MEDV'], axis=1)
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

## Task 2: Model Implementation & Scikit-Learn Design – 20%

In [ ]:
from sklearn.model_selection import cross_validate

#Function for cross validation testing for different models
def cross_test(model):
    # Cross-validation
    cross_val = cross_validate(estimator=model, X=x, y=y, cv=kfold, return_estimator=True, 
                               scoring=['neg_root_mean_squared_error', 'r2'], return_train_score=True)
    
    # Extract scores from the cross-validation results
    test_rmse_scores = -cross_val['test_neg_root_mean_squared_error']
    test_r2_scores = cross_val['test_r2']
    train_rmse_scores = -cross_val['train_neg_root_mean_squared_error']
    train_r2_scores = cross_val['train_r2']
    
    # Prepare the list of results
    result_list = []
    for i, model_ in enumerate(cross_val['estimator']):
        result_list.append({
            'fold': i+1, 
            'coefficients': model_.coef_, 
            'intercept': model_.intercept_,
            #Performance metrics: Test vs train RMSE, Test vs train R-squared
            'Test RMSE': test_rmse_scores[i],
            'Train RMSE': train_rmse_scores[i],
            'Test R²': test_r2_scores[i],
            'Train R²': train_r2_scores[i],
            'Average Test RMSE': test_rmse_scores.mean(),
            'Average Test R²': test_r2_scores.mean()
        })
    
    # Convert the result list to a DataFrame for better readability
    result_df = pd.DataFrame(result_list)
    return result_df, cross_val

#Function for retrieving model with the lowest RMSE -> best fit model
def best_coef(df, test_rmse_scores, cross_val):
   # Find the best model based on the lowest RMSE on the test set
    best_model_index = np.argmin(test_rmse_scores)
    
    # Get the row for the best model (corresponding to best_model_index)
    best_model_row = df.iloc[best_model_index]
    
    # Return coefficients and intercept for the best model
    best_coefficients = best_model_row['coefficients']
    intercept = best_model_row['intercept']

    feature_names = cross_val['estimator'][0].feature_names_in_
    
    # Add the intercept (constant) to the DataFrame
    coef_df = pd.DataFrame({
        'Coefficient': feature_names,
        'Value': best_coefficients
    })

    best_model = cross_val['estimator'][best_model_index]
    
    return coef_df, intercept, best_model_index, cross_val, best_model

#Function for plotting predicted y from model vs. tested y
def plotting(model):
    y_predict = model.predict(X_test)
    error = y_test - y_predict

    plt.figure(figsize=(10, 6))
    sns.kdeplot(y_test, label='y_test', color='blue', fill=True, alpha=0.5)
    sns.kdeplot(y_predict, label='y_predict', color='red', fill=True, alpha=0.5)
    sns.kdeplot(error, label='loss', color='green', fill=True, alpha=0.5)
    plt.title('Comparison of y_test and y_predict', fontsize=14)
    plt.xlabel('Value', fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.legend()
    plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

#Build Linear Regression Model
linearModel = LinearRegression()
linear_result_df, linear_cross_val = cross_test(linearModel)
linear_coef_df, linear_intercept, linear_best_model_index, linear_cross_val, linear_best_model = best_coef(linear_result_df, linear_result_df['Test RMSE'].values, linear_cross_val)

print("Best model: ")
print(linear_coef_df)
print("Intercept: ", linear_intercept)
linear_result_df

In [ ]:
# Plot actual vs predicted values
plotting(linear_best_model)

In [ ]:
from sklearn.linear_model import Ridge

# Build Ridge model with a given alpha
def ridge_model(alpha):
    return Ridge(alpha)

# Define a range of alpha values
alpha_range = np.arange(1, 11, 1)

# Initialize lists to store results and cross-validation data
all_results = []
all_cross_val = []

# Loop over the alpha values to train Ridge models
for i, alpha in enumerate(alpha_range):
    ridgeModel = ridge_model(alpha)
    
    # Cross-validation testing and model evaluation
    result_df, cross_val= cross_test(ridgeModel)
    
    result_df['Alpha'] = alpha
    all_results.append(result_df)
    all_cross_val.append(cross_val)

# Combine all results into a single dataframe
ridgeResult = pd.concat(all_results, ignore_index=True)
# Find the index of the best model
ridge_best_model_index = np.argmin(ridgeResult['Test RMSE'])
n_folds = len(all_cross_val[0]['estimator'])  #Number of models
alpha_index = ridge_best_model_index // n_folds  #Corresponsing alpha index of best model
fold_index = ridge_best_model_index % n_folds  # Corresponsing fold index of best model

# Retrieve the best model
ridge_best_model = all_cross_val[alpha_index]['estimator'][fold_index]
# Create a dataframe to store feature coefficients
ridge_coef_df = pd.DataFrame({
    'Coefficient': all_cross_val[alpha_index]['estimator'][0].feature_names_in_,  # Lấy tên các features
    'Value': ridge_best_model.coef_  # Lấy hệ số của mô hình
})
# Get the intercept
ridge_intercept = ridge_best_model.intercept_


print("Best model: ")
print(ridge_best_model)
print(ridge_coef_df)
print("Intercept: ", ridge_intercept)
ridgeResult

In [ ]:
# Plot actual vs predicted values
plotting(ridge_best_model)

In [ ]:
from sklearn.linear_model import SGDRegressor

# Build SGDRegressor model with specified hyperparameters
stochasticModel = SGDRegressor(
    alpha=0.0001, 
    eta0=0.01, 
    learning_rate='invscaling', 
    max_iter=1000)

# Perform cross-validation
sgd_result_df, sgd_cross_val = cross_test(stochasticModel)
# Retrieve the best coefficients, intercept, model index, and cross-validation data from the results
sgd_coef_df, sgd_intercept, sgd_best_model_index, sgd_cross_val, sgd_best_model = best_coef(sgd_result_df, sgd_result_df['Test RMSE'].values, sgd_cross_val)

print("Best model: ")
print(sgd_coef_df)
print("Intercept: ", sgd_intercept)
sgd_result_df

In [ ]:
# Plot actual vs predicted values
plotting(sgd_best_model)

## Task 3: Model Evaluation – 15%

After reviewing the results, it can be concluded that all three models yield relatively similar performance. Specifically:

* R² in the range (0.69 – 0.74): This indicates that the models can explain approximately 70% of the variance in the target variable, meaning they have relatively good predictive performance.
* RMSE in the range (4 – 6): The average prediction error falling within this range shows that the difference between the actual values and the predicted values is within an acceptable margin.